In [3]:
!pip install simpletransformers

In [4]:
from simpletransformers.classification import ClassificationModel
#?ClassificationModel
model = ClassificationModel('bert', 'hfl/chinese-bert-wwm')

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

In [5]:
!wget https://raw.githubusercontent.com/Moximixi/Emotional-analysis-of-curriculum-review/main/%E9%BB%84%E5%BF%A0%E8%B4%A2/newtest.csv

--2021-06-06 02:28:44--  https://raw.githubusercontent.com/Moximixi/Emotional-analysis-of-curriculum-review/main/%E9%BB%84%E5%BF%A0%E8%B4%A2/newtest.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 843461 (824K) [text/plain]
Saving to: ‘newtest.csv’

newtest.csv         100%[===================>] 823.69K  --.-KB/s    in 0.02s   

2021-06-06 02:28:44 (46.5 MB/s) - ‘newtest.csv’ saved [843461/843461]



In [6]:
import pandas as pd 
df = pd.read_csv("newtest.csv")
df=df.sample(frac=1.0)    #打乱数据集
df=df.reset_index(drop=True)
df[0:7500]


,comment,sentiment
0,理论知识是基础，但是能不能结合初中高中的案例加以分析呢,0
1,老师备课仔细，讲的特别认真,1
2,课程可以反复观看，非常方便，老师讲的很详细，生动，是非常好的课程。,1
3,荣老师讲得很有条理，相关的知识点讲得很详细，特别是针对难点，还请了实践经验丰富的专家来进行现...,1
4,太一般了，好像故意藏着掖着,0
...,...,...
7495,老师讲得很好。作为天文学的通识课值得所有专业学生学习。,1
7496,老师讲的和精彩,1
7497,本学期老师主要是通过口语授课和布置课堂作业的方式来教学。在学期的刚开始，老师对我们全体学生进...,1
7498,让我这个人力资源理论的小白，联系自己的工作环境与自身遇到的问题更加易于理解。很不错。,1


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train, test = train_test_split(df, test_size=.2, random_state=2)
train, valid = train_test_split(train, test_size=.2, random_state=2)
train.head()
#划分训练测试和验证集

,comment,sentiment
2041,课程各方面都挺好的,1
5492,非常棒，学习方便,1
7112,很棒，^ω^11,1
2620,很好很好，谢谢老师,1
3585,guna你可拉倒吧,0


In [9]:
model.train_model(train, args = {'overwrite_output_dir': True, 'learning_rate': 1e-5})

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/7241 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/906 [00:00<?, ?it/s]

(906, 0.28458926595992123)

In [10]:
result, model_output, wrong_predictions = model.eval_model(test)
result

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2263 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/283 [00:00<?, ?it/s]

{'auprc': 0.9530903399451189,
 'auroc': 0.9660233694846153,
 'eval_loss': 0.2701262428438416,
 'fn': 56,
 'fp': 86,
 'mcc': 0.8732263404135159,
 'tn': 933,
 'tp': 1188}

In [11]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np
Matrix = np.array([[result['tp'],result['fn']],[result['fp'],result['tn']]])
property_macro_list = []
TP_sum = 0
property_sum = sum(sum(Matrix))
Matrix_SumByCol = Matrix.sum(axis = 0)
for index in range(len(Matrix)):
    property_macro_list.append(Matrix[index][index] / Matrix_SumByCol[index])
    TP_sum += Matrix[index][index]
    
print("宏精确率:",sum(property_macro_list)/len(Matrix))
print("微精确率:",TP_sum/property_sum)

宏精确率: 0.9379366119941015
微精确率: 0.9372514361467079
